In [3]:

import snscrape.modules.twitter as sntwitter
import os
import datetime
import pandas as pd
df = pd.read_csv('./twibot-22.csv')
df['_id'] = df['id'].map(lambda x: x[1:])


In [4]:

justSome = True
final_df = pd.DataFrame()
for id, label in zip(df['_id'], df['label']):
    user_tweets_list = []

    list1 = []
    id = int(id)

    scrapper = sntwitter.TwitterProfileScraper(id)
    # print(scrapper.is_valid_username())
    try:
        user = scrapper.entity
        list1.append([user.username, user.id, label, user.displayname,
                      user.verified, user.created,
                      user.followersCount, user.friendsCount, user.statusesCount,
                      user.favouritesCount, user.listedCount, user.mediaCount, user.location,
                      user.protected, user.profileImageUrl])

    except:
        print('fail')
        continue

    user_df = pd.DataFrame(list1, columns=['username', 'id', 'label', 'displayname',
                                           'verified', 'created',
                                           'followersCount', 'friendsCount', 'statusesCount',
                                           'favouritesCount', 'listedCount', 'mediaCount', 'location',
                                           'protected', 'profileImageUrl'])
    current_time = datetime.datetime.now()
    for iteration, tweet in enumerate(sntwitter.TwitterUserScraper(id).get_items()):
        if (justSome == True):
            if (iteration > 100):
                break
            # print(tweet.)
        tweetItem = [tweet.date, tweet.user.username, tweet.lang,
                     tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount,
                     tweet.quoteCount, tweet.media, tweet.sourceLabel,  tweet.mentionedUsers]

        user_tweets_list.append(tweetItem)

    user_tweets_df = pd.DataFrame(user_tweets_list, columns=['DateTime',  'Username', 'Language',
                                                             'Hashtags', 'ReplyCount', 'RetweetCount', 'LikeCount',
                                                             'QuoteCount', 'Media', 'Source',  'mentionedUsers'])
    SourceList = user_tweets_df['Source'].unique().tolist()
    userDict = user_df.to_dict('records')[0]
    hashtagList = []
    mentionList = []
    for taglist in user_tweets_df['Hashtags']:
        if (taglist):
            for tag in taglist:
                hashtagList.append(tag)

    for mentions in user_tweets_df['mentionedUsers']:
        if (mentions):
            for mention in mentions:
                mentionList.append(mention.username)

    user_df['totalReplyCount'] = user_tweets_df['ReplyCount'].sum() or 0
    user_df['totalRetweetCount'] = user_tweets_df['RetweetCount'].sum() or 0
    user_df['totalLikeCount'] = user_tweets_df['LikeCount'].sum() or 0
    user_df['totalQuoteCount'] = user_tweets_df['QuoteCount'].sum() or 0
    user_df['totalSource'] = len(
        user_tweets_df['Source'].unique().tolist()) or 0

    user_df.to_csv('BotAccout.csv', mode='a', header=False, index=False)


fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


Unsupported unified_card type on tweet 1605192476318367744: 'twitter_article'


fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


Tweet 1611014657535262724 contains an app icon medium key '4_1611127364939042819' on app 'android_app'/'com.miHoYo.GenshinImpact', but the corresponding medium is missing; dropping
Tweet 1610652395549216769 contains an app icon medium key '4_1611127364939042819' on app 'android_app'/'com.miHoYo.GenshinImpact', but the corresponding medium is missing; dropping
Tweet 1610289890897297414 contains an app icon medium key '4_1611127364939042819' on app 'android_app'/'com.miHoYo.GenshinImpact', but the corresponding medium is missing; dropping
Tweet 1609927621084676097 contains an app icon medium key '4_1611127364939042819' on app 'android_app'/'com.miHoYo.GenshinImpact', but the corresponding medium is missing; dropping
Tweet 1609565246280110088 contains an app icon medium key '4_1611127364939042819' on app 'android_app'/'com.miHoYo.GenshinImpact', but the corresponding medium is missing; dropping
Tweet 1609203318466793472 contains an app icon medium key '4_1611127364939042819' on app 'andro

fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


Tweet 1591945597271117824 contains an app icon medium key '4_1611185483584790529' on app 'android_app'/'com.ussoccer.app_ussf_android', but the corresponding medium is missing; dropping


fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=from%3ACevherLIONS&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel%2ChasNftAvatar%2CvoiceInfo%2Cenrichments%2CsuperFollowMetadata%2CunmentionInfo: non-200 status code
4 requests to https://api.twitter.com/2/search/adaptive.json?include_profil

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=from%3ACevherLIONS&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&ext=mediaStats%2ChighlightedLabel%2ChasNftAvatar%2CvoiceInfo%2Cenrichments%2CsuperFollowMetadata%2CunmentionInfo failed, giving up.